In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [ ]:
import torch.nn as nn
from metal.mmtl.task import Task
from BERT_tasks import create_task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000)
bar_data = make_dataloader(2000)

foo = Task("foo_task", {"train":foo_data, "valid":foo_data, "test":None}, foo_input, foo_head, )
bar = create_task('SST2')
baz = create_task('COLA')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


 55%|█████▌    | 37319/67349 [00:08<00:06, 4326.45it/s]

In [ ]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

tasks = [foo,bar,baz]
model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=0.005, 
    progress_bar=True,
    log_every=0.2,
    log_unit="epochs",
    score_every=0.4,
    checkpoint_best=True,
    checkpoint_metric="SST2/accuracy",
    checkpoint_metric_mode="max"
)